# Answer Generator Node Test Notebook

This notebook tests the `generator.py` module functionality with various question types and contexts.

In [1]:
from dotenv import load_dotenv, find_dotenv

# .env 파일 로드
dotenv_path = find_dotenv()
load_dotenv(dotenv_path, override=True)

True

In [2]:
# Import required modules
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from app.agents.nodes.generator import generate_answer
from app.agents.prompts.generation import PROMPT_MAP, ERROR_MESSAGE
from app.agents.state import ChatState
import time

/Users/kimjunghyeon/Desktop/workspace/ai-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test Setup

Create mock chat states with different question types and contexts for testing

In [3]:
# Sample context for testing
SAMPLE_CONTEXTS = {
    "python_info": """
Python은 1991년 귀도 반 로섬(Guido van Rossum)이 개발한 고급 프로그래밍 언어입니다.
Python의 주요 특징:
- 간단하고 읽기 쉬운 문법
- 인터프리터 언어
- 객체지향 프로그래밍 지원
- 풍부한 라이브러리 생태계
- 크로스 플랫폼 지원

Python은 웹 개발, 데이터 분석, 머신러닝, 자동화 등 다양한 분야에서 사용됩니다.
""",
    
    "framework_comparison": """
Django vs FastAPI 비교:

Django:
- 2005년 출시
- Full-stack 웹 프레임워크
- ORM, 관리자 패널, 인증 시스템 내장
- MTV(Model-Template-View) 패턴
- 대규모 웹 애플리케이션에 적합

FastAPI:
- 2018년 출시
- 고성능 API 프레임워크
- 자동 API 문서화 (Swagger/OpenAPI)
- Type hints 기반 검증
- 비동기 처리 최적화
- REST API 개발에 특화
""",
    
    "machine_learning": """
머신러닝(Machine Learning)은 인공지능의 한 분야로, 컴퓨터가 명시적으로 프로그래밍되지 않고도
데이터로부터 학습하여 패턴을 찾고 예측을 수행하는 기술입니다.

주요 유형:
1. 지도학습 (Supervised Learning): 레이블된 데이터로 학습
   - 예시: 이미지 분류, 스팸 메일 필터링
2. 비지도학습 (Unsupervised Learning): 레이블 없는 데이터로 패턴 발견
   - 예시: 클러스터링, 이상 탐지
3. 강화학습 (Reinforcement Learning): 환경과의 상호작용을 통한 학습
   - 예시: 게임 AI, 로봇 제어

Python의 대표적인 머신러닝 라이브러리:
- scikit-learn: 전통적 머신러닝 알고리즘
- TensorFlow: 구글에서 개발한 딥러닝 프레임워크
- PyTorch: 페이스북에서 개발한 딥러닝 프레임워크
- pandas: 데이터 조작 및 분석
- numpy: 수치 계산
"""
}

def create_test_state(question_type, messages, context="", user_id="test_user", session_id="test_session"):
    """테스트용 ChatState 생성"""
    return {
        "messages": messages,
        "question_type": question_type,
        "context": context,
        "user_id": user_id,
        "session_id": session_id,
        "thread_id": f"thread_{session_id}",
        "model_used": ""
    }

print("Test setup completed!")
print(f"Available prompt types: {list(PROMPT_MAP.keys())}")
print(f"Error message: {ERROR_MESSAGE}")

Test setup completed!
Available prompt types: ['FACT', 'SUMMARY', 'COMPARE', 'EVIDENCE']
Error message: 답변 생성 중 오류가 발생했습니다. 다시 시도해주세요.


## Test Case 1: FACT Questions

Test generation of factual answers with context

In [4]:
async def test_fact_generation():
    print("=== FACT Question Generation Test ===")
    
    test_cases = [
        {
            "description": "Basic Python information",
            "messages": [HumanMessage(content="Python이란 무엇인가요?", id="msg1")],
            "context": SAMPLE_CONTEXTS["python_info"]
        },
        {
            "description": "Python features inquiry",
            "messages": [
                AIMessage(content="안녕하세요!", id="msg1"),
                HumanMessage(content="Python의 주요 특징은 무엇인가요?", id="msg2")
            ],
            "context": SAMPLE_CONTEXTS["python_info"]
        },
        {
            "description": "Question without context",
            "messages": [HumanMessage(content="JavaScript란 무엇인가요?", id="msg1")],
            "context": ""
        }
    ]
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {test_case['description']}")
        
        # Create test state
        state = create_test_state(
            question_type="FACT",
            messages=test_case["messages"],
            context=test_case["context"]
        )
        
        # Show input info
        last_human_msg = next(
            (msg.content for msg in reversed(test_case["messages"]) if isinstance(msg, HumanMessage)),
            "No question found"
        )
        print(f"  Question: {last_human_msg}")
        print(f"  Context available: {'Yes' if test_case['context'] else 'No'}")
        print(f"  Context length: {len(test_case['context'])} chars")
        
        # Generate answer
        try:
            start_time = time.time()
            result = await generate_answer(state)
            end_time = time.time()
            
            answer = result.get("answer", "")
            model_used = result.get("model_used", "")
            messages = result.get("messages", [])
            
            print(f"  Model: {model_used}")
            print(f"  Generation time: {end_time - start_time:.2f}s")
            print(f"  Answer length: {len(answer)} chars")
            print(f"  Answer preview: {answer[:200]}{'...' if len(answer) > 200 else ''}")
            print(f"  Messages returned: {len(messages)}")
            
            # Check if it's an error message
            if answer == ERROR_MESSAGE:
                print(f"  ⚠️ Error response detected")
            else:
                print(f"  ✅ Answer generated successfully")
                
        except Exception as e:
            print(f"  ❌ Error: {e}")

await test_fact_generation()

=== FACT Question Generation Test ===

Test 1: Basic Python information
  Question: Python이란 무엇인가요?
  Context available: Yes
  Context length: 200 chars
  Model: gpt-4o
  Generation time: 1.41s
  Answer length: 194 chars
  Answer preview: Python은 1991년 귀도 반 로섬(Guido van Rossum)이 개발한 고급 프로그래밍 언어입니다. 주요 특징으로는 간단하고 읽기 쉬운 문법, 인터프리터 언어, 객체지향 프로그래밍 지원, 풍부한 라이브러리 생태계, 크로스 플랫폼 지원 등이 있습니다. Python은 웹 개발, 데이터 분석, 머신러닝, 자동화 등 다양한 분야에서 사용됩니다.
  Messages returned: 1
  ✅ Answer generated successfully

Test 2: Python features inquiry
  Question: Python의 주요 특징은 무엇인가요?
  Context available: Yes
  Context length: 200 chars
  Model: gpt-4o
  Generation time: 1.02s
  Answer length: 141 chars
  Answer preview: Python의 주요 특징은 다음과 같습니다:

- 간단하고 읽기 쉬운 문법
- 인터프리터 언어
- 객체지향 프로그래밍 지원
- 풍부한 라이브러리 생태계
- 크로스 플랫폼 지원

이러한 특징들 덕분에 Python은 다양한 분야에서 널리 사용되고 있습니다.
  Messages returned: 1
  ✅ Answer generated successfully

Test 3: Question without context
  Question: JavaScript란 무엇인가요?
  Context available: No
  Context lengt

## Test Case 2: SUMMARY Questions

Test generation of summary responses

In [5]:
async def test_summary_generation():
    print("=== SUMMARY Question Generation Test ===")
    
    test_cases = [
        {
            "description": "Summarize Python information",
            "messages": [HumanMessage(content="Python에 대해 요약해주세요", id="msg1")],
            "context": SAMPLE_CONTEXTS["python_info"]
        },
        {
            "description": "Summarize machine learning",
            "messages": [HumanMessage(content="머신러닝에 대한 내용을 정리해주세요", id="msg1")],
            "context": SAMPLE_CONTEXTS["machine_learning"]
        },
        {
            "description": "Summary without context",
            "messages": [HumanMessage(content="블록체인 기술을 요약해주세요", id="msg1")],
            "context": ""
        }
    ]
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {test_case['description']}")
        
        state = create_test_state(
            question_type="SUMMARY",
            messages=test_case["messages"],
            context=test_case["context"]
        )
        
        last_human_msg = next(
            (msg.content for msg in reversed(test_case["messages"]) if isinstance(msg, HumanMessage)),
            "No question found"
        )
        print(f"  Question: {last_human_msg}")
        print(f"  Context available: {'Yes' if test_case['context'] else 'No'}")
        
        try:
            start_time = time.time()
            result = await generate_answer(state)
            end_time = time.time()
            
            answer = result.get("answer", "")
            model_used = result.get("model_used", "")
            
            print(f"  Model: {model_used}")
            print(f"  Generation time: {end_time - start_time:.2f}s")
            print(f"  Answer length: {len(answer)} chars")
            print(f"  Answer preview: {answer[:300]}{'...' if len(answer) > 300 else ''}")
            
            if answer == ERROR_MESSAGE:
                print(f"  ⚠️ Error response detected")
            else:
                print(f"  ✅ Summary generated successfully")
                
        except Exception as e:
            print(f"  ❌ Error: {e}")

await test_summary_generation()

=== SUMMARY Question Generation Test ===

Test 1: Summarize Python information
  Question: Python에 대해 요약해주세요
  Context available: Yes
  Model: gpt-4o
  Generation time: 2.39s
  Answer length: 241 chars
  Answer preview: Python은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다. 주요 특징은 다음과 같습니다:

- **문법**: 간단하고 읽기 쉬운 문법
- **언어 유형**: 인터프리터 언어
- **프로그래밍 패러다임**: 객체지향 프로그래밍 지원
- **라이브러리**: 풍부한 라이브러리 생태계
- **호환성**: 크로스 플랫폼 지원

Python은 웹 개발, 데이터 분석, 머신러닝, 자동화 등 다양한 분야에서 널리 사용됩니다.
  ✅ Summary generated successfully

Test 2: Summarize machine learning
  Question: 머신러닝에 대한 내용을 정리해주세요
  Context available: Yes
  Model: gpt-4o
  Generation time: 2.46s
  Answer length: 456 chars
  Answer preview: 머신러닝은 인공지능의 한 분야로, 컴퓨터가 데이터로부터 학습하여 패턴을 찾고 예측을 수행하는 기술입니다.

주요 유형:
1. 지도학습 (Supervised Learning)
   - 레이블된 데이터로 학습
   - 예시: 이미지 분류, 스팸 메일 필터링

2. 비지도학습 (Unsupervised Learning)
   - 레이블 없는 데이터로 패턴 발견
   - 예시: 클러스터링, 이상 탐지

3. 강화학습 (Reinforcement Learning)
   - 환경과의 상호작용을 통한 학습
   - 예시: 게임 AI, 로봇 제어

...
  ✅ Summary generate

## Test Case 3: COMPARE Questions

Test generation of comparison responses

In [6]:
async def test_compare_generation():
    print("=== COMPARE Question Generation Test ===")
    
    test_cases = [
        {
            "description": "Django vs FastAPI comparison",
            "messages": [HumanMessage(content="Django와 FastAPI를 비교해주세요", id="msg1")],
            "context": SAMPLE_CONTEXTS["framework_comparison"]
        },
        {
            "description": "Programming language comparison",
            "messages": [
                HumanMessage(content="안녕하세요", id="msg1"),
                AIMessage(content="안녕하세요! 무엇을 도와드릴까요?", id="msg2"),
                HumanMessage(content="Python과 Java의 차이점은 무엇인가요?", id="msg3")
            ],
            "context": SAMPLE_CONTEXTS["python_info"]
        },
        {
            "description": "Comparison without sufficient context",
            "messages": [HumanMessage(content="React와 Vue.js 중 어떤 것이 더 좋나요?", id="msg1")],
            "context": ""
        }
    ]
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {test_case['description']}")
        
        state = create_test_state(
            question_type="COMPARE",
            messages=test_case["messages"],
            context=test_case["context"]
        )
        
        last_human_msg = next(
            (msg.content for msg in reversed(test_case["messages"]) if isinstance(msg, HumanMessage)),
            "No question found"
        )
        print(f"  Question: {last_human_msg}")
        print(f"  Context available: {'Yes' if test_case['context'] else 'No'}")
        
        try:
            start_time = time.time()
            result = await generate_answer(state)
            end_time = time.time()
            
            answer = result.get("answer", "")
            model_used = result.get("model_used", "")
            
            print(f"  Model: {model_used}")
            print(f"  Generation time: {end_time - start_time:.2f}s")
            print(f"  Answer length: {len(answer)} chars")
            print(f"  Answer preview: {answer[:300]}{'...' if len(answer) > 300 else ''}")
            
            if answer == ERROR_MESSAGE:
                print(f"  ⚠️ Error response detected")
            else:
                print(f"  ✅ Comparison generated successfully")
                
        except Exception as e:
            print(f"  ❌ Error: {e}")

await test_compare_generation()

=== COMPARE Question Generation Test ===

Test 1: Django vs FastAPI comparison
  Question: Django와 FastAPI를 비교해주세요
  Context available: Yes
  Model: gpt-4o
  Generation time: 8.76s
  Answer length: 1713 chars
  Answer preview: Django와 FastAPI는 Python으로 작성된 웹 프레임워크로, 각기 다른 장점과 특성을 가지고 있습니다. 아래 표를 통해 두 프레임워크의 공통점과 차이점을 비교하겠습니다:

| 특징               | Django                                   | FastAPI                                       |
|--------------------|------------------------------------------|---------------------...
  ✅ Comparison generated successfully

Test 2: Programming language comparison
  Question: Python과 Java의 차이점은 무엇인가요?
  Context available: Yes
  Model: gpt-4o
  Generation time: 6.83s
  Answer length: 1765 chars
  Answer preview: Python과 Java는 모두 널리 사용되는 프로그래밍 언어지만, 여러 면에서 차이가 있습니다. 아래 표를 통해 그 차이점과 공통점을 비교해 보겠습니다.

| 특징                | Python                                                      | Java                                                        |
|------

## Test Case 4: EVIDENCE Questions

Test generation of evidence-based responses

In [ ]:
async def test_evidence_generation():
    print("=== EVIDENCE Question Generation Test ===")
    
    test_cases = [
        {
            "description": "Evidence for Python popularity",
            "messages": [HumanMessage(content="Python이 인기 있는 이유의 근거를 제시해주세요", id="msg1")],
            "context": SAMPLE_CONTEXTS["python_info"]
        },
        {
            "description": "Evidence for machine learning effectiveness",
            "messages": [HumanMessage(content="머신러닝이 효과적인 이유를 증거와 함께 설명해주세요", id="msg1")],
            "context": SAMPLE_CONTEXTS["machine_learning"]
        },
        {
            "description": "Evidence request without context",
            "messages": [HumanMessage(content="클라우드 컴퓨팅이 필요한 근거는 무엇인가요?", id="msg1")],
            "context": ""
        }
    ]
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {test_case['description']}")
        
        state = create_test_state(
            question_type="EVIDENCE",
            messages=test_case["messages"],
            context=test_case["context"]
        )
        
        last_human_msg = next(
            (msg.content for msg in reversed(test_case["messages"]) if isinstance(msg, HumanMessage)),
            "No question found"
        )
        print(f"  Question: {last_human_msg}")
        print(f"  Context available: {'Yes' if test_case['context'] else 'No'}")
        
        try:
            start_time = time.time()
            result = await generate_answer(state)
            end_time = time.time()
            
            answer = result.get("answer", "")
            model_used = result.get("model_used", "")
            
            print(f"  Model: {model_used}")
            print(f"  Generation time: {end_time - start_time:.2f}s")
            print(f"  Answer length: {len(answer)} chars")
            print(f"  Answer preview: {answer[:300]}{'...' if len(answer) > 300 else ''}")
            
            if answer == ERROR_MESSAGE:
                print(f"  ⚠️ Error response detected")
            else:
                print(f"  ✅ Evidence-based answer generated successfully")
                
        except Exception as e:
            print(f"  ❌ Error: {e}")

await test_evidence_generation()

## Test Case 5: Edge Cases and Error Handling

Test various edge cases and error scenarios

In [ ]:
async def test_edge_cases():
    print("=== Edge Cases and Error Handling Test ===")
    
    test_cases = [
        {
            "description": "Empty messages",
            "question_type": "FACT",
            "messages": [],
            "context": SAMPLE_CONTEXTS["python_info"]
        },
        {
            "description": "Invalid question type",
            "question_type": "INVALID_TYPE",
            "messages": [HumanMessage(content="What is AI?", id="msg1")],
            "context": SAMPLE_CONTEXTS["machine_learning"]
        },
        {
            "description": "None question type (should default to FACT)",
            "question_type": None,
            "messages": [HumanMessage(content="Python이란?", id="msg1")],
            "context": SAMPLE_CONTEXTS["python_info"]
        },
        {
            "description": "Empty context and messages",
            "question_type": "SUMMARY",
            "messages": [],
            "context": ""
        },
        {
            "description": "Very long context",
            "question_type": "FACT",
            "messages": [HumanMessage(content="이 내용에 대해 알려주세요", id="msg1")],
            "context": SAMPLE_CONTEXTS["machine_learning"] * 10  # Very long context
        }
    ]
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {test_case['description']}")
        
        state = create_test_state(
            question_type=test_case["question_type"],
            messages=test_case["messages"],
            context=test_case["context"]
        )
        
        print(f"  Question type: {test_case['question_type']}")
        print(f"  Messages count: {len(test_case['messages'])}")
        print(f"  Context length: {len(test_case['context'])} chars")
        
        try:
            start_time = time.time()
            result = await generate_answer(state)
            end_time = time.time()
            
            answer = result.get("answer", "")
            model_used = result.get("model_used", "")
            messages = result.get("messages", [])
            
            print(f"  Model: {model_used}")
            print(f"  Generation time: {end_time - start_time:.2f}s")
            print(f"  Answer length: {len(answer)} chars")
            print(f"  Messages returned: {len(messages)}")
            
            if answer == ERROR_MESSAGE:
                print(f"  ⚠️ Error response as expected")
            elif answer:
                print(f"  ✅ Response generated")
                print(f"  Answer preview: {answer[:150]}{'...' if len(answer) > 150 else ''}")
            else:
                print(f"  ❌ Empty response")
                
        except Exception as e:
            print(f"  ❌ Exception: {e}")

await test_edge_cases()

## Test Case 6: Prompt Selection Logic

Test that the correct prompt is selected for each question type

In [ ]:
async def test_prompt_selection():
    print("=== Prompt Selection Logic Test ===")
    
    # Test each question type to ensure correct prompt usage
    question_types = ["FACT", "SUMMARY", "COMPARE", "EVIDENCE"]
    
    for question_type in question_types:
        print(f"\nTesting {question_type} prompt selection:")
        
        state = create_test_state(
            question_type=question_type,
            messages=[HumanMessage(content=f"{question_type} 타입 질문입니다", id="msg1")],
            context=SAMPLE_CONTEXTS["python_info"]
        )
        
        try:
            result = await generate_answer(state)
            answer = result.get("answer", "")
            model_used = result.get("model_used", "")
            
            print(f"  Question type: {question_type}")
            print(f"  Model: {model_used}")
            print(f"  Answer generated: {'Yes' if answer and answer != ERROR_MESSAGE else 'No'}")
            print(f"  Answer length: {len(answer)} chars")
            
            # Check if prompt was selected correctly (indirect test)
            if question_type in PROMPT_MAP:
                print(f"  ✅ Valid prompt type used")
            else:
                print(f"  ⚠️ Fallback to FACT prompt expected")
                
        except Exception as e:
            print(f"  ❌ Error: {e}")

await test_prompt_selection()

## Performance and Quality Analysis

Analyze generation performance and quality

In [ ]:
async def performance_analysis():
    print("=== Performance and Quality Analysis ===")
    
    # Test multiple generations for timing analysis
    test_state = create_test_state(
        question_type="FACT",
        messages=[HumanMessage(content="Python의 특징을 설명해주세요", id="msg1")],
        context=SAMPLE_CONTEXTS["python_info"]
    )
    
    print("Running 5 consecutive generations for performance analysis...")
    
    generation_times = []
    answer_lengths = []
    
    for i in range(5):
        try:
            start_time = time.time()
            result = await generate_answer(test_state)
            end_time = time.time()
            
            generation_time = end_time - start_time
            answer = result.get("answer", "")
            
            generation_times.append(generation_time)
            answer_lengths.append(len(answer))
            
            print(f"  Run {i+1}: {generation_time:.2f}s, {len(answer)} chars")
            
        except Exception as e:
            print(f"  Run {i+1}: Error - {e}")
    
    if generation_times:
        avg_time = sum(generation_times) / len(generation_times)
        min_time = min(generation_times)
        max_time = max(generation_times)
        
        avg_length = sum(answer_lengths) / len(answer_lengths)
        min_length = min(answer_lengths)
        max_length = max(answer_lengths)
        
        print(f"\nPerformance Summary:")
        print(f"  Average generation time: {avg_time:.2f}s")
        print(f"  Min generation time: {min_time:.2f}s")
        print(f"  Max generation time: {max_time:.2f}s")
        print(f"  Average answer length: {avg_length:.0f} chars")
        print(f"  Min answer length: {min_length} chars")
        print(f"  Max answer length: {max_length} chars")
        
        # Performance assessment
        if avg_time < 3.0:
            print(f"  ✅ Good performance (< 3s average)")
        elif avg_time < 5.0:
            print(f"  ⚠️ Acceptable performance (3-5s average)")
        else:
            print(f"  ❌ Slow performance (> 5s average)")

await performance_analysis()

## Manual Test: Interactive Generation

Test custom questions and contexts interactively

In [ ]:
async def test_custom_generation(question, question_type="FACT", context=""):
    """
    Test generation with custom input
    """
    print(f"Custom Generation Test:")
    print(f"  Question: {question}")
    print(f"  Type: {question_type}")
    print(f"  Context: {'Provided' if context else 'None'}")
    
    state = create_test_state(
        question_type=question_type,
        messages=[HumanMessage(content=question, id="custom")],
        context=context
    )
    
    try:
        start_time = time.time()
        result = await generate_answer(state)
        end_time = time.time()
        
        answer = result.get("answer", "")
        model_used = result.get("model_used", "")
        
        print(f"  Model: {model_used}")
        print(f"  Generation time: {end_time - start_time:.2f}s")
        print(f"  Answer:")
        print(f"  {answer}")
        
    except Exception as e:
        print(f"  Error: {e}")

# Example custom tests
print("=== Custom Generation Tests ===")

# Test 1: Fact question with context
await test_custom_generation(
    question="Python이 데이터 과학에 인기 있는 이유는 무엇인가요?",
    question_type="FACT",
    context=SAMPLE_CONTEXTS["machine_learning"]
)

print("\n" + "-"*50 + "\n")

# Test 2: Summary without context
await test_custom_generation(
    question="클라우드 컴퓨팅에 대해 요약해주세요",
    question_type="SUMMARY",
    context=""
)

## Test Results Summary

Overall test summary and recommendations

In [ ]:
print("\n" + "="*60)
print("GENERATOR MODULE TEST SUMMARY")
print("="*60)

print("\nKey Test Areas Covered:")
print("  ✓ FACT question generation")
print("  ✓ SUMMARY question generation")
print("  ✓ COMPARE question generation")
print("  ✓ EVIDENCE question generation")
print("  ✓ Edge cases and error handling")
print("  ✓ Prompt selection logic")
print("  ✓ Performance analysis")
print("  ✓ Custom generation testing")

print("\nPrompt Types Available:")
for prompt_type in PROMPT_MAP.keys():
    print(f"  - {prompt_type}")

print("\nRecommendations for Production:")
print("  1. Monitor generation times and implement timeout handling")
print("  2. Add response quality validation")
print("  3. Implement retry logic for failed generations")
print("  4. Add context length validation and truncation")
print("  5. Consider caching for frequently asked questions")
print("  6. Add metrics collection for model performance")

print("\n" + "="*60)